In [23]:
import pandas as pd
import numpy as np

In [ ]:
df_tasas_productos = pd.read_excel('./datos/tasas_productos.xlsx')
df_obligaciones_clientes = pd.read_excel('./datos/obligaciones_clientes.xlsx')

In [24]:
merged_df = df_obligaciones_clientes.merge(df_tasas_productos, 
                                           left_on=['cod_segm_tasa', 'cod_subsegm_tasa', 'cal_interna_tasa'], 
                                           right_on=['cod_segmento', 'cod_subsegmento', 'calificacion_riesgos'])


In [25]:
def obtener_tasa(row):
    if 'cartera' in row['id_producto'] and 'leasing' not in row['id_producto']:
        return row['tasa_cartera']
    elif 'operacion_especifica' in row['id_producto']:
        return row['tasa_operacion_especifica']
    elif 'hipotecario' in row['id_producto']:
        return row['tasa_hipotecario']
    elif 'leasing' in row['id_producto']:
        return row['tasa_leasing']
    elif 'sufi' in row['id_producto']:
        return row['tasa_sufi']
    elif 'factoring' in row['id_producto']:
        return row['tasa_factoring']
    elif 'tarjeta' in row['id_producto']:
        return row['tasa_tarjeta']
    else:
        return 0

In [26]:

def obtener_valor_periodicidad(periodicidad):
    if periodicidad == 'MENSUAL':
        return 12
    elif periodicidad == 'BIMENSUAL':
        return 6
    elif periodicidad == 'TRIMESTRAL':
        return 4
    elif periodicidad == 'SEMESTRAL':
        return 2
    elif periodicidad == 'ANUAL':
        return 1
    else:
        return 1

In [27]:

    
def calcular_tasa_efectiva(row):
    return (1 + row['tasa'])**(1 / obtener_valor_periodicidad(row['periodicidad'])) - 1

In [28]:
def calcular_valor_final(row):
    return np.power(1 + row['tasa_efectiva'], row['plazo']) * row['valor_inicial']

In [29]:
merged_df['tasa'] = merged_df.apply(obtener_tasa, axis=1)


merged_df['tasa_efectiva'] = merged_df.apply(calcular_tasa_efectiva, axis=1)
merged_df['valor_final'] = merged_df.apply(calcular_valor_final, axis=1)

final_df = merged_df[['radicado', 'num_documento', 'id_producto', 'valor_inicial', 'tasa', 'periodicidad', 'tasa_efectiva', 'valor_final']]

C:\Users\Rocío\AppData\Local\Temp\ipykernel_23460\2228733901.py:2: RuntimeWarning: overflow encountered in power
  return np.power(1 + row['tasa_efectiva'], row['plazo']) * row['valor_inicial']


In [ ]:
final_df

In [32]:
df_sumas_clientes = merged_df.groupby('num_documento').agg(
    suma_valor_final=('valor_final', 'sum'),
    cantidad_productos=('id_producto', 'nunique')
)

df_sumas_clientes = df_sumas_clientes[df_sumas_clientes['cantidad_productos'] >= 2]

df_sumas_clientes.reset_index(inplace=True)

df_sumas_clientes

,num_documento,suma_valor_final,cantidad_productos
0,3907738,4.681834e+05,8
1,4234923,1.027217e+06,8
2,4935132,1.201929e+06,7
3,7061608,1.382985e+06,4
4,7099543,1.088517e+06,6
...,...,...,...
242,1339938146,2.989122e+05,2
243,1342616595,5.456629e+05,6
244,1344463827,1.036454e+06,4
245,1346212202,1.678844e+06,5
